# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/bidding_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/binfo_V2-5158000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-999000
INFO:tensorflow:Re

### Bidding

In [2]:
vuln_ns, vuln_ew = False, False
hand = 'AQJ986.QJ952.7.8'

auction = ["PASS", "1N"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 2, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '2C', 'insta_score': 0.999}]

In [3]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


West always has the same cards because it's your hand and it's known to you

In [4]:
# we got some samples above. if we want more, we can always get more
auction = ["PASS", "1N"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction, 2)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')

Good quality: True
xxx.8xx.T8x.Axxx KTx.AKxx.AQx.KJx AQJ98x.QJ9xx.x.8 x.T.KJ9xxx.QT9xx 0.75
xxxx.xxx.K98x.AT Kx.A8.AJTxx.K9xx AQJ98x.QJ9xx.x.8 T.KTx.Qxx.QJxxxx 1.00
KTxx.Txx.Jxx.Jxx x.AK.A8xxx.KQ9xx AQJ98x.QJ9xx.x.8 xx.8xx.KQT9.ATxx 0.75
xxx.ATx.Jxx.xxxx KTx.xx.A98x.ATxx AQJ98x.QJ9xx.x.8 x.K8x.KQTxx.KQJ9 0.75
Txx.xxxx.98xx.Qx Kx.K.AKQ.AKJTxxx AQJ98x.QJ9xx.x.8 xx.AT8.JTxxx.9xx 0.75
xx.xx.KT8x.J9xxx Kxx.KT.AQ9x.QTxx AQJ98x.QJ9xx.x.8 Tx.A8xx.Jxxx.AKx 0.75
KTx.x.KQTxx.QT9x xx.KTxx.A8x.AKxx AQJ98x.QJ9xx.x.8 xx.A8x.J9xx.Jxxx 0.75
Kxx.Tx.T8xxx.QJx Txxx.Kxx.Qx.AKTx AQJ98x.QJ9xx.x.8 .A8x.AKJ9x.9xxxx 0.75
xxx.Ax.AJ9xx.J9x Kx.T.Qxxx.AKTxxx AQJ98x.QJ9xx.x.8 Tx.K8xxx.KT8.Qxx 0.75
Txx.KT.Axx.QT9xx Kx.Axx.8xxx.AKJx AQJ98x.QJ9xx.x.8 xx.8xx.KQJT9.xxx 1.00
